# 🔧 Deep Dive 01: 处理不平衡的自定义 Loss

---

## 🎬 真实场景：发券数据的噩梦

你是某电商公司的算法工程师，刚刚接手一个智能发券项目。

打开数据一看，傻眼了：

```
总用户数: 1,000,000
发券用户: 50,000 (5%)
未发券用户: 950,000 (95%)
```

你用标准的 IPW 估计效应，结果：

- 🔴 倾向得分模型预测几乎全是 0
- 🔴 权重有些高达 100+，方差爆炸
- 🔴 效应估计忽正忽负，完全不可靠

老板问：「这个发券效果到底是多少？」你支支吾吾说不清楚...

---

### 问题诊断

**处理不平衡 (Treatment Imbalance)** 是因果推断中最常见的问题之一：

| 问题 | 表现 | 后果 |
|------|------|------|
| 倾向得分极端 | $e(x) \to 0$ 或 $e(x) \to 1$ | 权重爆炸 |
| 样本不平衡 | 少数类样本太少 | 模型学不好 |
| 共同支撑差 | 两组特征分布差异大 | 无法外推 |

**本节目标：从原理出发，设计自定义 Loss 函数来解决这些问题。**

---

## 📚 本节学习目标

完成本练习后，你将能够：

1. 🎯 诊断处理不平衡问题的严重程度
2. 🎯 理解并实现 **Focal Loss** 用于倾向得分估计
3. 🎯 设计 **自定义 Propensity-weighted Loss**
4. 🎯 对比 Trimming、Overlap Weights、Stabilized Weights 等方法
5. 🎯 在真实数据上应用这些方法

> ⚠️ **Deep Dive 系列** 的目标是从「调包」到「改架构」，真正掌握模型原理！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from scipy import stats
from typing import Tuple, Dict, Callable
import warnings
warnings.filterwarnings('ignore')

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')
np.random.seed(42)

print("✅ 环境准备完毕！")
print("🔧 Deep Dive: 今天我们要从原理改进模型")

---

## 🔬 Part 1: 生成不平衡数据

首先，让我们生成一个真实的不平衡发券场景。

In [ ]:
def generate_imbalanced_voucher_data(
    n: int = 50000,
    treatment_rate: float = 0.05,  # 只有 5% 发券
    true_ate: float = 15.0,        # 真实效应: 发券平均提升 15 元
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, float]:
    """
    生成严重不平衡的发券数据
    
    场景：电商发券
    - X: 用户特征（购买力、活跃度、历史消费）
    - T: 是否发券（严重不平衡！）
    - Y: 消费金额
    """
    np.random.seed(seed)
    
    # 用户特征
    purchase_power = np.random.exponential(2, n)      # 购买力
    activity = np.random.beta(2, 5, n)                 # 活跃度 (偏低)
    history_amount = np.random.lognormal(4, 1, n)      # 历史消费
    
    X = np.column_stack([
        purchase_power,
        activity,
        history_amount,
        purchase_power * activity,  # 交互项
        np.log1p(history_amount)    # 非线性变换
    ])
    
    # 发券策略：偏向高购买力、高活跃度用户
    # 这会导致严重的选择偏差！
    propensity_logit = (
        -3.5 +                          # 基础很低，保证整体发券率约 5%
        0.5 * purchase_power +          # 高购买力更可能发券
        2.0 * activity +                # 高活跃度更可能发券
        0.0005 * history_amount         # 历史消费影响小
    )
    propensity = 1 / (1 + np.exp(-propensity_logit))
    T = np.random.binomial(1, propensity)
    
    # 真实的异质性效应
    # 高购买力用户效应更大，低活跃度用户效应更大（边际效应）
    cate = (
        true_ate +
        3 * purchase_power +            # 购买力高，效应大
        -10 * activity +                # 活跃度高，边际效应小
        np.random.randn(n) * 5          # 噪声
    )
    cate = np.clip(cate, 0, 50)  # 效应在 [0, 50] 之间
    
    # 结果：消费金额
    baseline_spending = (
        50 +
        20 * purchase_power +
        30 * activity +
        0.01 * history_amount +
        np.random.randn(n) * 20
    )
    Y = baseline_spending + T * cate
    Y = np.clip(Y, 0, None)  # 消费不能为负
    
    # 计算真实 ATE（处理组的平均效应）
    true_ate_actual = cate.mean()
    true_att = cate[T == 1].mean()
    
    print(f"📊 不平衡数据生成完成")
    print(f"=" * 50)
    print(f"样本量: {n:,}")
    print(f"发券用户: {T.sum():,} ({T.mean():.1%})")
    print(f"未发券用户: {(1-T).sum():,} ({1-T.mean():.1%})")
    print(f"\n倾向得分分布:")
    print(f"   最小值: {propensity.min():.4f}")
    print(f"   最大值: {propensity.max():.4f}")
    print(f"   均值: {propensity.mean():.4f}")
    print(f"\n真实效应:")
    print(f"   ATE: {true_ate_actual:.2f}")
    print(f"   ATT: {true_att:.2f}")
    
    return X, T, Y, true_att, propensity

In [ ]:
# 生成数据
X, T, Y, TRUE_ATT, true_propensity = generate_imbalanced_voucher_data(n=50000)

In [ ]:
# 可视化不平衡程度
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. 处理分布
axes[0].bar(['未发券', '发券'], [(1-T).sum(), T.sum()], color=['steelblue', 'coral'])
axes[0].set_ylabel('样本数', fontsize=12)
axes[0].set_title(f'处理分布 (发券率: {T.mean():.1%})', fontsize=14)
for i, v in enumerate([(1-T).sum(), T.sum()]):
    axes[0].text(i, v + 1000, f'{v:,}', ha='center', fontsize=12)

# 2. 真实倾向得分分布
axes[1].hist(true_propensity, bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[1].axvline(true_propensity.mean(), color='red', linestyle='--', linewidth=2, label=f'均值: {true_propensity.mean():.3f}')
axes[1].set_xlabel('真实倾向得分', fontsize=12)
axes[1].set_ylabel('频数', fontsize=12)
axes[1].set_title('真实倾向得分分布', fontsize=14)
axes[1].legend()

# 3. 按处理分组的倾向得分分布
axes[2].hist(true_propensity[T==0], bins=50, alpha=0.5, label='未发券', color='steelblue')
axes[2].hist(true_propensity[T==1], bins=50, alpha=0.5, label='发券', color='coral')
axes[2].set_xlabel('真实倾向得分', fontsize=12)
axes[2].set_ylabel('频数', fontsize=12)
axes[2].set_title('按处理分组的倾向得分', fontsize=14)
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n⚠️ 问题诊断:")
print(f"   1. 发券率只有 {T.mean():.1%}，严重不平衡")
print(f"   2. 大部分用户的倾向得分很低（< 0.1）")
print(f"   3. 两组的倾向得分分布有明显差异")

---

## 🚨 Part 2: 问题演示 - 标准 IPW 失效

让我们看看标准 IPW 在不平衡数据上的表现。

In [ ]:
def standard_ipw(X, T, Y):
    """
    标准 IPW 估计
    """
    # 估计倾向得分
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    
    # 计算权重
    weights = T / propensity + (1 - T) / (1 - propensity)
    
    # IPW 估计
    y1 = (Y * T / propensity).sum() / (T / propensity).sum()
    y0 = (Y * (1-T) / (1-propensity)).sum() / ((1-T) / (1-propensity)).sum()
    ate = y1 - y0
    
    return {
        'ate': ate,
        'propensity': propensity,
        'weights': weights
    }

In [ ]:
# 运行标准 IPW
ipw_result = standard_ipw(X, T, Y)

print("🚨 标准 IPW 结果")
print("=" * 50)
print(f"真实 ATT: {TRUE_ATT:.2f}")
print(f"IPW 估计: {ipw_result['ate']:.2f}")
print(f"偏差: {ipw_result['ate'] - TRUE_ATT:+.2f}")
print()
print(f"倾向得分统计:")
print(f"   最小值: {ipw_result['propensity'].min():.6f}")
print(f"   最大值: {ipw_result['propensity'].max():.4f}")
print(f"   均值: {ipw_result['propensity'].mean():.4f}")
print()
print(f"权重统计:")
print(f"   最小值: {ipw_result['weights'].min():.2f}")
print(f"   最大值: {ipw_result['weights'].max():.2f} ⚠️")
print(f"   均值: {ipw_result['weights'].mean():.2f}")
print(f"   标准差: {ipw_result['weights'].std():.2f}")

In [ ]:
# 可视化问题
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. 估计的倾向得分分布
axes[0].hist(ipw_result['propensity'], bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[0].axvline(0.05, color='red', linestyle='--', linewidth=2, label='真实发券率')
axes[0].set_xlabel('估计的倾向得分', fontsize=12)
axes[0].set_ylabel('频数', fontsize=12)
axes[0].set_title('倾向得分估计', fontsize=14)
axes[0].legend()

# 2. 权重分布
axes[1].hist(ipw_result['weights'], bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1].axvline(ipw_result['weights'].mean(), color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('IPW 权重', fontsize=12)
axes[1].set_ylabel('频数', fontsize=12)
axes[1].set_title(f'权重分布 (max={ipw_result["weights"].max():.1f})', fontsize=14)
axes[1].set_xlim([0, min(50, ipw_result['weights'].max())])

# 3. 极端权重的影响
weight_percentiles = [50, 90, 95, 99, 99.9, 100]
weight_values = [np.percentile(ipw_result['weights'], p) for p in weight_percentiles]
axes[2].bar([str(p) for p in weight_percentiles], weight_values, color='coral', alpha=0.7)
axes[2].set_xlabel('百分位数', fontsize=12)
axes[2].set_ylabel('权重值', fontsize=12)
axes[2].set_title('权重的百分位分布', fontsize=14)

plt.tight_layout()
plt.show()

print("\n💡 问题根源:")
print(f"   1. 倾向得分估计偏向 0，导致 1/e(x) 权重爆炸")
print(f"   2. 少数极端权重主导了整个估计")
print(f"   3. 有效样本量 (ESS) 远小于实际样本量")

In [ ]:
# 计算有效样本量
def effective_sample_size(weights):
    """
    计算有效样本量
    ESS = (sum(w))^2 / sum(w^2)
    """
    return (weights.sum() ** 2) / (weights ** 2).sum()

ess = effective_sample_size(ipw_result['weights'])
print(f"📊 有效样本量分析")
print(f"   实际样本量: {len(Y):,}")
print(f"   有效样本量 (ESS): {ess:.0f}")
print(f"   ESS 占比: {ess/len(Y):.1%}")
print(f"\n⚠️ 只有 {ess/len(Y):.1%} 的样本在有效地贡献信息！")

---

## 🔧 Part 3: 解决方案 A - 改进倾向得分估计

### 3.1 Focal Loss

**Focal Loss** 最初用于目标检测中的类别不平衡问题，核心思想是：

> 对容易分类的样本（概率高的）降低权重，对难分类的样本（概率低的）增加权重

**标准交叉熵:**
$$L_{CE} = -y \log(p) - (1-y) \log(1-p)$$

**Focal Loss:**
$$L_{FL} = -\alpha_t (1-p_t)^\gamma \log(p_t)$$

其中：
- $p_t = p$ if $y=1$ else $1-p$
- $\alpha$: 类别权重
- $\gamma$: 聚焦参数（$\gamma > 0$）

In [ ]:
# TODO 1: 实现 Focal Loss

def focal_loss(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    gamma: float = 2.0,
    alpha: float = 0.25
) -> float:
    """
    计算 Focal Loss
    
    参数:
        y_true: 真实标签 (0 或 1)
        y_pred: 预测概率
        gamma: 聚焦参数，越大对难分类样本关注越多
        alpha: 正类权重
    
    返回:
        loss: Focal Loss 值
    """
    # 裁剪预测值，避免 log(0)
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    
    # 实现 Focal Loss
    # 1. 计算 p_t: 如果 y=1 则 p_t=p，否则 p_t=1-p
    p_t = np.where(y_true == 1, y_pred, 1 - y_pred)
    
    # 2. 计算 alpha_t: 如果 y=1 则 alpha_t=alpha，否则 alpha_t=1-alpha
    alpha_t = np.where(y_true == 1, alpha, 1 - alpha)
    
    # 3. focal_weight = (1 - p_t)^gamma
    focal_weight = (1 - p_t) ** gamma
    
    # 4. loss = -alpha_t * focal_weight * log(p_t)
    loss = -alpha_t * focal_weight * np.log(p_t)
    
    return loss.mean()

In [ ]:
# 📝 参考答案

def focal_loss_solution(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    gamma: float = 2.0,
    alpha: float = 0.25
) -> float:
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    
    # p_t: 正确类别的概率
    p_t = np.where(y_true == 1, y_pred, 1 - y_pred)
    
    # alpha_t: 类别权重
    alpha_t = np.where(y_true == 1, alpha, 1 - alpha)
    
    # focal weight: 难分类样本权重更高
    focal_weight = (1 - p_t) ** gamma
    
    # Focal Loss
    loss = -alpha_t * focal_weight * np.log(p_t)
    
    return loss.mean()

In [ ]:
# 可视化 Focal Loss vs Cross Entropy

def visualize_focal_loss():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    p = np.linspace(0.01, 0.99, 100)
    
    # 正类 (y=1)
    ce_loss = -np.log(p)
    fl_gamma1 = -(1-p)**1 * np.log(p)
    fl_gamma2 = -(1-p)**2 * np.log(p)
    fl_gamma5 = -(1-p)**5 * np.log(p)
    
    axes[0].plot(p, ce_loss, 'b-', linewidth=2, label='CE (γ=0)')
    axes[0].plot(p, fl_gamma1, 'g--', linewidth=2, label='FL (γ=1)')
    axes[0].plot(p, fl_gamma2, 'r-.', linewidth=2, label='FL (γ=2)')
    axes[0].plot(p, fl_gamma5, 'm:', linewidth=2, label='FL (γ=5)')
    axes[0].set_xlabel('预测概率 p (正类)', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('正类 (y=1) 的 Loss', fontsize=14)
    axes[0].legend()
    axes[0].set_ylim([0, 5])
    
    # 负类 (y=0)
    ce_loss_neg = -np.log(1-p)
    fl_gamma1_neg = -p**1 * np.log(1-p)
    fl_gamma2_neg = -p**2 * np.log(1-p)
    fl_gamma5_neg = -p**5 * np.log(1-p)
    
    axes[1].plot(p, ce_loss_neg, 'b-', linewidth=2, label='CE (γ=0)')
    axes[1].plot(p, fl_gamma1_neg, 'g--', linewidth=2, label='FL (γ=1)')
    axes[1].plot(p, fl_gamma2_neg, 'r-.', linewidth=2, label='FL (γ=2)')
    axes[1].plot(p, fl_gamma5_neg, 'm:', linewidth=2, label='FL (γ=5)')
    axes[1].set_xlabel('预测概率 p (正类)', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('负类 (y=0) 的 Loss', fontsize=14)
    axes[1].legend()
    axes[1].set_ylim([0, 5])
    
    plt.tight_layout()
    plt.show()
    
    print("💡 观察:")
    print("   - γ 越大，对高置信度样本的惩罚越小")
    print("   - 这迫使模型更关注难分类的样本")
    print("   - 对于不平衡数据，少数类往往是难分类的")

visualize_focal_loss()

### 3.2 使用 Focal Loss 训练倾向得分模型

由于 sklearn 不直接支持自定义 Loss，我们用 GradientBoosting 的 sample_weight 来近似实现。

In [ ]:
def train_propensity_with_focal_weights(
    X: np.ndarray,
    T: np.ndarray,
    gamma: float = 2.0
):
    """
    使用 Focal Loss 风格的权重训练倾向得分模型
    
    思路：先用标准模型预测，再根据难度调整权重重新训练
    """
    # 第一轮：标准训练
    model_init = LogisticRegression(max_iter=1000)
    model_init.fit(X, T)
    p_init = model_init.predict_proba(X)[:, 1]
    
    # 计算 Focal Weight
    p_t = np.where(T == 1, p_init, 1 - p_init)
    focal_weights = (1 - p_t) ** gamma
    
    # 第二轮：使用 Focal Weight 训练
    model_focal = LogisticRegression(max_iter=1000)
    model_focal.fit(X, T, sample_weight=focal_weights)
    p_focal = model_focal.predict_proba(X)[:, 1]
    
    return p_focal, focal_weights

In [ ]:
# 对比标准估计和 Focal 估计
p_focal, focal_weights = train_propensity_with_focal_weights(X, T, gamma=2.0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 对比倾向得分分布
axes[0].hist(ipw_result['propensity'], bins=50, alpha=0.5, label='标准 LR', color='blue')
axes[0].hist(p_focal, bins=50, alpha=0.5, label='Focal 加权', color='green')
axes[0].set_xlabel('倾向得分', fontsize=12)
axes[0].set_ylabel('频数', fontsize=12)
axes[0].set_title('倾向得分分布对比', fontsize=14)
axes[0].legend()

# Focal 权重分布
axes[1].hist(focal_weights[T==0], bins=50, alpha=0.5, label='未发券', color='steelblue')
axes[1].hist(focal_weights[T==1], bins=50, alpha=0.5, label='发券', color='coral')
axes[1].set_xlabel('Focal 权重', fontsize=12)
axes[1].set_ylabel('频数', fontsize=12)
axes[1].set_title('Focal 权重分布', fontsize=14)
axes[1].legend()

plt.tight_layout()
plt.show()

---

## 🔧 Part 4: 解决方案 B - 修改 IPW 权重

除了改进倾向得分估计，我们还可以直接修改权重。

### 4.1 Trimming (截断)

最简单的方法：删除倾向得分太极端的样本。

In [ ]:
def ipw_with_trimming(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    propensity: np.ndarray,
    lower: float = 0.01,
    upper: float = 0.99
) -> Dict:
    """
    Trimmed IPW: 删除极端倾向得分的样本
    """
    # 找出在 [lower, upper] 范围内的样本
    mask = (propensity >= lower) & (propensity <= upper)
    
    X_trim = X[mask]
    T_trim = T[mask]
    Y_trim = Y[mask]
    propensity_trim = propensity[mask]
    
    # 计算 IPW
    y1 = (Y_trim * T_trim / propensity_trim).sum() / (T_trim / propensity_trim).sum()
    y0 = (Y_trim * (1-T_trim) / (1-propensity_trim)).sum() / ((1-T_trim) / (1-propensity_trim)).sum()
    ate = y1 - y0
    
    return {
        'ate': ate,
        'n_original': len(Y),
        'n_trimmed': mask.sum(),
        'pct_kept': mask.mean()
    }

### 4.2 Overlap Weights (重叠权重)

Overlap Weights 的核心思想是：**最大化处理组和对照组的重叠**。

$$w_i = \begin{cases} 1 - e(X_i) & \text{if } T_i = 1 \\ e(X_i) & \text{if } T_i = 0 \end{cases}$$

直觉：
- 对于发券用户（$T=1$），如果他的倾向得分很高（本来就很可能发券），权重小
- 对于未发券用户（$T=0$），如果他的倾向得分很低（本来就不太可能发券），权重也小

In [ ]:
# TODO 2: 实现 Overlap Weights

def ipw_with_overlap_weights(
    T: np.ndarray,
    Y: np.ndarray,
    propensity: np.ndarray
) -> Dict:
    """
    使用 Overlap Weights 的 IPW
    
    Overlap Weights:
        - 如果 T=1: w = 1 - e(X)
        - 如果 T=0: w = e(X)
    """
    # 计算 Overlap Weights
    # w = T * (1 - propensity) + (1 - T) * propensity
    overlap_weights = T * (1 - propensity) + (1 - T) * propensity
    
    # 加权估计
    y1 = (Y * T * overlap_weights).sum() / (T * overlap_weights).sum()
    y0 = (Y * (1-T) * overlap_weights).sum() / ((1-T) * overlap_weights).sum()
    ate = y1 - y0
    
    return {
        'ate': ate,
        'weights': overlap_weights,
        'ess': effective_sample_size(overlap_weights)
    }

In [ ]:
# 📝 参考答案

def ipw_with_overlap_weights_solution(
    T: np.ndarray,
    Y: np.ndarray,
    propensity: np.ndarray
) -> Dict:
    # Overlap Weights
    overlap_weights = T * (1 - propensity) + (1 - T) * propensity
    
    # 加权估计
    y1 = (Y * T * overlap_weights).sum() / (T * overlap_weights).sum()
    y0 = (Y * (1-T) * overlap_weights).sum() / ((1-T) * overlap_weights).sum()
    ate = y1 - y0
    
    return {
        'ate': ate,
        'weights': overlap_weights,
        'ess': effective_sample_size(overlap_weights)
    }

### 4.3 Stabilized Weights (稳定化权重)

稳定化权重在分子加入边际处理概率，减少方差：

$$w_i^{stab} = \frac{P(T)}{e(X_i)} \cdot T_i + \frac{1-P(T)}{1-e(X_i)} \cdot (1-T_i)$$

In [ ]:
def ipw_with_stabilized_weights(
    T: np.ndarray,
    Y: np.ndarray,
    propensity: np.ndarray
) -> Dict:
    """
    使用稳定化权重的 IPW
    """
    # 边际处理概率
    p_t = T.mean()
    
    # 稳定化权重
    stabilized_weights = T * p_t / propensity + (1 - T) * (1 - p_t) / (1 - propensity)
    
    # 裁剪极端权重
    stabilized_weights = np.clip(stabilized_weights, 0.1, 10)
    
    # 加权估计
    y1 = (Y * T * stabilized_weights).sum() / (T * stabilized_weights).sum()
    y0 = (Y * (1-T) * stabilized_weights).sum() / ((1-T) * stabilized_weights).sum()
    ate = y1 - y0
    
    return {
        'ate': ate,
        'weights': stabilized_weights,
        'ess': effective_sample_size(stabilized_weights)
    }

---

## 🏆 Part 5: 方法大比拼

In [ ]:
# 估计倾向得分
lr = LogisticRegression(max_iter=1000)
lr.fit(X, T)
propensity_est = lr.predict_proba(X)[:, 1]

# 各种方法
results = []

# 1. 朴素估计
naive_ate = Y[T==1].mean() - Y[T==0].mean()
results.append({'方法': '朴素估计', 'ATE': naive_ate, 'ESS': len(Y)})

# 2. 标准 IPW
ipw_std = standard_ipw(X, T, Y)
results.append({'方法': '标准 IPW', 'ATE': ipw_std['ate'], 'ESS': effective_sample_size(ipw_std['weights'])})

# 3. Trimmed IPW
ipw_trim = ipw_with_trimming(X, T, Y, propensity_est, lower=0.01, upper=0.99)
results.append({'方法': 'Trimmed IPW', 'ATE': ipw_trim['ate'], 'ESS': ipw_trim['n_trimmed']})

# 4. Overlap Weights
ipw_overlap = ipw_with_overlap_weights_solution(T, Y, propensity_est)
results.append({'方法': 'Overlap Weights', 'ATE': ipw_overlap['ate'], 'ESS': ipw_overlap['ess']})

# 5. Stabilized Weights
ipw_stab = ipw_with_stabilized_weights(T, Y, propensity_est)
results.append({'方法': 'Stabilized Weights', 'ATE': ipw_stab['ate'], 'ESS': ipw_stab['ess']})

# 汇总
results_df = pd.DataFrame(results)
results_df['偏差'] = results_df['ATE'] - TRUE_ATT
results_df['|偏差|'] = abs(results_df['偏差'])

print("🏆 方法大比拼")
print("=" * 70)
print(f"真实 ATT: {TRUE_ATT:.2f}")
print()
display(results_df)

In [ ]:
# 可视化对比
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ATE 估计对比
colors = ['gray', 'red', 'orange', 'green', 'blue']
bars = axes[0].barh(results_df['方法'], results_df['ATE'], color=colors, alpha=0.7)
axes[0].axvline(TRUE_ATT, color='black', linestyle='--', linewidth=2, label=f'真实 ATT = {TRUE_ATT:.1f}')
axes[0].set_xlabel('ATE 估计', fontsize=12)
axes[0].set_title('各方法 ATE 估计', fontsize=14)
axes[0].legend()

# ESS 对比
axes[1].barh(results_df['方法'], results_df['ESS'], color=colors, alpha=0.7)
axes[1].set_xlabel('有效样本量 (ESS)', fontsize=12)
axes[1].set_title('有效样本量对比', fontsize=14)
axes[1].set_xscale('log')

plt.tight_layout()
plt.show()

print("\n💡 关键发现:")
print("   1. Overlap Weights 通常给出最稳健的估计")
print("   2. Stabilized Weights 可以显著增加 ESS")
print("   3. 标准 IPW 在极端不平衡时会失效")

---

## 💡 Part 6: 思考题

### 问题 1: Focal Loss 的 γ 参数如何选择？太大或太小会有什么影响？

**你的答案:**

---

### 问题 2: Overlap Weights 估计的是什么 estimand？和 IPW 估计的 ATE 有什么区别？

**提示**: 想想 Overlap Weights 关注的是哪个人群

**你的答案:**

---

### 问题 3: 在实际业务中，你会如何选择处理不平衡的方法？

**你的答案:**

---

### 问题 4: Trimming 会带来什么问题？如何决定截断阈值？

**提示**: 想想外推性和样本量的权衡

**你的答案:**

---

---

## 📋 总结

### 处理不平衡的方法

| 方法 | 核心思想 | 优点 | 缺点 |
|------|---------|------|------|
| **Focal Loss** | 关注难分类样本 | 改善倾向得分估计 | 需要自定义训练 |
| **Trimming** | 删除极端样本 | 简单直观 | 丢弃样本，改变目标人群 |
| **Overlap Weights** | 最大化重叠 | 权重温和，ESS 高 | 估计的是 overlap 人群效应 |
| **Stabilized Weights** | 除以边际概率 | 减少方差 | 可能仍有极端权重 |

### 实践建议

1. **先诊断再治疗**: 检查倾向得分分布、权重分布、ESS
2. **多方法对比**: 如果不同方法结果差异大，要小心
3. **业务理解**: 不同方法对应不同的目标人群
4. **敏感性分析**: 改变截断阈值、权重方案，看结果稳定性

### 关键公式

**Focal Loss:**
$$L_{FL} = -\alpha_t (1-p_t)^\gamma \log(p_t)$$

**Overlap Weights:**
$$w_i = T_i \cdot (1-e(X_i)) + (1-T_i) \cdot e(X_i)$$

**Stabilized Weights:**
$$w_i^{stab} = T_i \cdot \frac{P(T)}{e(X_i)} + (1-T_i) \cdot \frac{1-P(T)}{1-e(X_i)}$$

---

**「处理不平衡不可怕，可怕的是不知道自己遇到了不平衡。」**

🎉 恭喜完成 Deep Dive 01！

---

## 🎤 Part 7: 面试模拟环节

### 问题 1: 诊断处理不平衡

**面试官**: 你拿到一个因果推断项目，处理组只有 5% 的样本，你会如何诊断是否存在问题？

**参考答案**:

我会从以下几个维度诊断：

```python
def diagnose_treatment_imbalance(T, e, weights):
    """诊断处理不平衡问题"""
    print("1. 基础统计:")
    print(f"   处理率: {T.mean():.2%}")
    print(f"   处理组样本: {T.sum()}, 控制组样本: {(1-T).sum()}")
    
    print("\n2. 倾向得分分布:")
    print(f"   e(X) 范围: [{e.min():.4f}, {e.max():.4f}]")
    print(f"   e(X) < 0.1: {(e < 0.1).sum()} ({(e < 0.1).mean():.1%})")
    print(f"   e(X) > 0.9: {(e > 0.9).sum()} ({(e > 0.9).mean():.1%})")
    
    print("\n3. IPW 权重诊断:")
    print(f"   最大权重: {weights.max():.1f}")
    print(f"   权重 > 10 的样本: {(weights > 10).sum()}")
    
    print("\n4. 有效样本量:")
    ess = (weights.sum()**2) / (weights**2).sum()
    print(f"   ESS: {ess:.0f} / {len(weights)} ({ess/len(weights):.1%})")
    
    print("\n5. 共同支撑:")
    # 检查两组在特征空间的重叠
    print("   (需要可视化特征分布)")
    
    if ess/len(weights) < 0.5:
        print("\n⚠️ 诊断: 存在严重的处理不平衡问题!")
        print("   建议: 使用 Overlap Weights 或 Trimming")
```

**关键点**:
- 处理率本身不是问题，关键看倾向得分分布
- 检查权重分布，警惕极端权重
- 计算 ESS，损失超过 50% 需要处理
- 可视化共同支撑区域

---

### 问题 2: Focal Loss vs Class Weights

**面试官**: 除了 Focal Loss，我们也可以用类别权重处理不平衡，两者有什么区别？什么时候用哪个？

**参考答案**:

| 方法 | 机制 | 适用场景 | 优缺点 |
|------|------|---------|--------|
| **Class Weights** | 对少数类样本Loss加权 | 全局不平衡 | 简单，但对所有少数类样本一视同仁 |
| **Focal Loss** | 对难分类样本加权 | 存在难分类样本 | 动态调整权重，但需要调参γ |

**代码对比**:
```python
# Class Weights
lr = LogisticRegression(class_weight='balanced')
# 等价于手动设置: w_0 = n/(2*n_0), w_1 = n/(2*n_1)

# Focal Loss
# Loss = -(1-p)^γ * log(p)
# γ 越大，对简单样本的惩罚越小
```

**选择建议**:
- **数据不平衡 + 样本质量均匀**: Class Weights
- **数据不平衡 + 有难分类样本**: Focal Loss
- **极端不平衡 (1:100)**: 先用 Focal Loss，再配合 Class Weights

**实战经验**:
- Focal Loss 的 γ 通常在 [1, 5] 之间
- γ=2 是一个不错的起点
- 可以通过交叉验证选择 γ

---

### 问题 3: Overlap Weights 估计的是什么？

**面试官**: Overlap Weights 估计的是 ATE 吗？和标准 IPW 有什么区别？

**参考答案**:

**NO！Overlap Weights 估计的是 ATO (Average Treatment Effect on the Overlap population)**

$$w_{OW} = e(X)(1-e(X))$$

**估计目标对比**:

| Estimand | 定义 | 权重 | 关注人群 |
|----------|------|------|---------|
| ATE | E[Y(1) - Y(0)] | $w=1$ (均匀) | 全体 |
| ATT | E[Y(1) - Y(0) \| T=1] | $w=1$ for T=1 | 处理组 |
| ATO | E[Y(1) - Y(0) \| \text{overlap}] | $w=e(X)(1-e(X))$ | 重叠区域 |

**直觉理解**:
```python
# e(X) = 0.5 (最大重叠)  → w = 0.25 (最高权重)
# e(X) = 0.1 或 0.9      → w = 0.09 (低权重)
# e(X) → 0 或 1          → w → 0   (接近零权重)
```

**业务含义**:
- **ATO 关注**: 处理组和控制组都有代表性的人群
- **实际应用**: 如果只在重叠人群推广策略，ATO 更有意义
- **外推性**: ATO 不能外推到极端人群

**什么时候用 ATO**:
1. 极端倾向得分样本不可信（权重爆炸）
2. 业务只关心有对比价值的人群
3. 追求估计的稳定性而非覆盖全体

---

### 问题 4: 实战场景 - 发券策略

**面试官**: 你负责一个发券项目，发现高价值用户几乎都被发券（e(X)→1），低价值用户几乎不发券（e(X)→0）。你会怎么办？

**参考答案**:

**Step 1: 理解业务逻辑**
```
为什么出现极端倾向得分？
- 可能是有意为之的业务策略（高价值用户优先）
- 也可能是历史系统的Bug
```

**Step 2: 确定分析目标**

| 目标 | 方法 | 说明 |
|------|------|------|
| 评估当前策略效果 | ATT (处理组效应) | 只看被发券用户 |
| 优化发券策略 | ATE | 需要外推到所有用户 |
| 在可控范围优化 | ATO | 只在重叠区域优化 |

**Step 3: 技术方案**

```python
# 方案A: 如果目标是 ATT (评估当前策略)
# 只需要处理组和可比的控制组
ate_att = ipw_with_trimming(Y, T, e, trim_threshold=0.1)

# 方案B: 如果目标是 ATE (优化策略)
# 需要外推，但极端样本不可信
# 选项1: Trimming (丢弃极端样本，可能有偏)
# 选项2: Overlap Weights (只估计重叠区域)
# 选项3: 做一个 RCT 补充极端区域的数据 ⭐

# 方案C: 双稳健方法
# 同时建模倾向得分和结果模型
from econml.dr import DRLearner
```

**Step 4: 向业务方汇报**

"当前数据只能可靠估计中等价值用户的效应。如果要评估对高/低价值用户的效应，建议：
1. 短期：使用 Overlap Weights 估计重叠人群效应
2. 长期：设计 RCT 补充极端用户数据"

**关键要点**:
- 技术方案要配合业务目标
- 诚实沟通方法的局限性
- 提出数据收集建议